# Get ecg signal from dataset, Call PanTompkins to write peaks to output.txt

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import wfdb
import os 
import subprocess
os.chdir ("C:/Users/ARTAPC/Desktop/EcgSamples")
FILE= '100'
signals, fields= wfdb.rdsamp(FILE)
ssignal=signals[:,0][0:500000]
Exefile_Path = "C:/Users/ARTAPC/Desktop/Git/test/src"   # Path to where PanTompkins.exe is located.
os.chdir(Exefile_Path)

with open('ecg.txt', 'w') as f:
    for line in ssignal:
        f.write(f"{line}\n")

subprocess.call(r"PanTompkins.exe", stdin=None, stdout=None, stderr=None, shell=False) # Calls PanTompkins.exe to write in output.txt
lines =[]
with open("output.txt", 'r') as file:
    # Iterate over each line in the file
    for line in file:
        # Strip any extra whitespace or newline characters
        stripped_line = line.strip()
        # Convert the line to a number (assuming integers)
       
        number = float(stripped_line)  # Use float to handle both integers and decimals
        lines.append(number)

## Combine ecg.txt and output.txt into one file where each line is --> sample,0/1 (depending on whether the sample is a peak or not)

In [3]:
ecg_file = 'ecg.txt'
peaks_file = 'output.txt'
output_file = 'ecg_with_peaks.txt'

# Read ECG samples
with open(ecg_file, 'r') as f:
    ecg_samples = [line.strip() for line in f]

# Read peak line numbers
with open(peaks_file, 'r') as f:
    peak_lines = set(int(line.strip()) for line in f)

    

# Write the output file
with open(output_file, 'w') as f:
    for i, sample in enumerate(ecg_samples):
        peak_flag = 1 if (i + 1) in peak_lines else 0
        f.write(f"{sample},{peak_flag}\n")

# Down sampling signals for being able to show them with less delay # (Optional)

In [4]:

ecg_samples = np.array(ecg_samples) 
PealLinesList = list(sorted(peak_lines))
peak_indices =np.array(PealLinesList) 

# Downsampling function
def downsample_ecg(ecg, target_length):
    current_length = len(ecg)
    if current_length <= target_length:
        return ecg
    
    indices = np.linspace(0, current_length - 1, target_length, dtype=int)
    return ecg[indices]

# Downsample the ECG samples
downsampled_ecg = downsample_ecg(ecg_samples, 400000).tolist()




# To downsample the peak indices accordingly, we can apply the same downsampling logic
downsampled_peak_indices = [int(i*(400000/500000)) for i in peak_indices]


downsampledsignal = 'downsampledsignal.txt'
downsampledpeaks = 'downsampledpeaks.txt'

with open(downsampledsignal, 'w') as file:
    # Iterate over each element in the list
    for element in downsampled_ecg:
        # Write the element followed by a newline
        file.write(f"{element}\n")


with open(downsampledpeaks, 'w') as file:
    # Iterate over each element in the list
    for element in downsampled_peak_indices:
        # Write the element followed by a newline
        file.write(f"{element}\n")


Exefile_Path = "C:/Users/ARTAPC/Desktop/Git/test/src"   # Path to where PanTompkins.exe is located.
os.chdir(Exefile_Path)
ecg_file = 'downsampledsignal.txt'
peaks_file = 'downsampledpeaks.txt'
output_file = 'ecg_with_peaks_downsampled.txt'

# Read ECG samples
with open(ecg_file, 'r') as f:
    ecg_samples = [line.strip() for line in f]

# Read peak line numbers
with open(peaks_file, 'r') as f:
    peak_lines = set(int(line.strip()) for line in f)

    

# Write the output file
with open(output_file, 'w') as f:
    for i, sample in enumerate(ecg_samples):
        peak_flag = 1 if (i + 1) in peak_lines else 0
        f.write(f"{sample},{peak_flag}\n")

'''
peaks = np.zeros(len(downsampled_ecg))

for peak_index in downsampled_peak_indices:
     # Ensure the peak index is within the valid range
    peaks[peak_index] = 1


peaks=peaks.astype(int)

#rpeaks = peaks.tolist()

#rpeaks[657]

'''

'\npeaks = np.zeros(len(downsampled_ecg))\n\nfor peak_index in downsampled_peak_indices:\n     # Ensure the peak index is within the valid range\n    peaks[peak_index] = 1\n\n\npeaks=peaks.astype(int)\n\n#rpeaks = peaks.tolist()\n\n#rpeaks[657]\n\n'

In [1]:
import sys
import serial
import pyqtgraph as pg
from PyQt5.QtWidgets import QApplication, QMainWindow
from PyQt5.QtCore import QTimer

class ECGPlotter(QMainWindow):
    def __init__(self, serial_port, *args, **kwargs):
        super(ECGPlotter, self).__init__(*args, **kwargs)
        self.setWindowTitle('ECG Signal Plotter')
        self.setGeometry(100, 100, 800, 600)

        # Create a plot widget
        self.plotWidget = pg.PlotWidget()
        self.setCentralWidget(self.plotWidget)

        # Initialize the data
        self.data = []
        self.ptr = 0

        # Set up the plot
        self.plotWidget.setLabel('left', 'ECG Signal')
        self.plotWidget.setLabel('bottom', 'Sample Index')
        self.plotWidget.showGrid(True, True, 0.5)
        self.curve = self.plotWidget.plot(pen='r')

        # Timer to update the plot
        self.timer = QTimer()
        self.timer.timeout.connect(self.updatePlot)
        self.timer.start(0)  # Update every 1 ms

        # Use the passed serial port
        self.ser = serial_port

        # ECG signal data and peaks passed as arguments
        #self.ecg_samples = ecg_samples
        #self.peaks = peaks

        self.current_index = 0
        self.peak_lines = []  # To keep track of the peak lines

    def updatePlot(self):
        if self.current_index < 500000:
            # Prepare data line with sample and peak status
            #sample = self.ecg_samples[self.current_index]
            #is_peak = self.peaks[self.current_index]
            #data_line = f"{sample},{is_peak}\n"

            # Write the line to COM port
            #self.ser.write(data_line.encode())

            # Read the line from COM port
            line = self.ser.readline().decode().strip()
            try:
                read_sample, read_is_peak = line.split(',')
                read_sample = float(read_sample)
                read_is_peak = int(read_is_peak)

                # Append data and update plot
                self.data.append(read_sample)
                self.ptr += 1
                self.curve.setData(self.data)

                # Update peak markers with vertical lines
                if read_is_peak == 1:
                    peak_line = pg.InfiniteLine(pos=self.ptr + 5 - 1, angle=90, pen=pg.mkPen(color='g', width=2))
                    self.plotWidget.addItem(peak_line)
                    self.peak_lines.append(peak_line)
                    
            except ValueError:
                pass  # Handle any parsing errors

            # Update plot range
            self.plotWidget.setXRange(max(0, self.ptr - 100), self.ptr)
            
            self.current_index += 1
        else:
            self.timer.stop()

    def closeEvent(self, event):
        """Override closeEvent to properly close serial port and stop timer."""
        print("Window is closing")
        if self.ser.is_open:
            self.ser.close()  # Close the serial port immediately
            print("Serial port closed")
        self.timer.stop()  # Stop the timer
        event.accept()  # Accept the event to close the window



# Initialize the serial port outside of the ECGPlotter class

# Initialize the serial port and run the application
serial_port = serial.Serial('COM4', baudrate=38400, timeout=1,parity=serial.PARITY_EVEN,rtscts=False, xonxoff=True, dsrdtr=False)

#ecg_samples = downsampled_ecg  # Replace with actual ECG sample data
app = QApplication(sys.argv)
window = ECGPlotter(serial_port, )
window.show()
sys.exit(app.exec_())


Window is closing
Serial port closed


SystemExit: 0

c:\Users\ARTAPC\miniconda3\envs\py38\lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
